OSM export
==========
Loads a street graph and exports it in OSM format, e.g., for MATSim

In [5]:
import snman

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

In [6]:
# =====================================================================================
# LOAD GRAPH
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

Load street graph


In [7]:
len(G.edges)

102662

In [8]:
# =====================================================================================
# EXPORT OSM
# =====================================================================================

print('Export OSM file')
export_osm_tags = {
        'highway', 'maxspeed',
        'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways', 'oneway',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        '_connected_component'
    }

snman.io.export_osm_xml(G, process_path + 'osm.osm', export_osm_tags, uv_tags=True, tag_all_nodes=False)

Export OSM file
